In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

plt.style.use('fivethirtyeight')
plt.rcParams['figure.figsize'] = (10, 8)
plt.rcParams['font.size'] = 14

from sklearn import metrics
from sklearn.metrics import confusion_matrix

In [6]:
full_df = pd.read_csv('../../data/full_df.csv',index_col=False)
user_products = pd.read_csv('../../data/user_products.csv',index_col=False)

## Baseline Model: Simple Prediction
The baseline I want to establish is predicting if a user will reorder a product based on his/her most recent order

In [ ]:
# #get unique user-product where the user_id is in the train set 
# orders = user_products[user_products['user_id'].isin(user_products[user_products['eval_set']=='train']['user_id'])] \
#                            .drop_duplicates(['user_id', 'product_id'])
# # orders.drop('Unnamed: 0',axis=1,inplace=True)

In [ ]:
#get unique user-product where the user_id is in the train set 
# test  = user_products[user_products['eval_set'] == 'train' ]
# user_ids = test['user_id'].values
# orders = user_products[user_products['user_id'].isin(user_ids)]

In [14]:
full_df.target.value_counts(normalize=True)

0    0.9022
1    0.0978
Name: target, dtype: float64

In [24]:
full_df

,user_id,product_id,eval_set,target,order_id,user_product_count,user_product_first_order,user_product_last_order,user_product_avg_basket_placement,product_ordered_vol,...,user_reorder_ratio,user_total_orders,user_lifetime_days,user_avg_days_between_orders,user_max_time_between_orders,user_min_time_between_orders,user_avg_cart_size,user_product_order_rate,user_product_reorder_rate,user_product_last_time_product_ordered
0,112108,49302,train,1,1,3,1,4,2.000000,170,...,0.764706,4,167.0,9.823529,15.0,7.0,6.250000,0.750000,0.750000,0
1,112108,11109,train,1,1,3,1,4,3.333333,4579,...,0.764706,4,167.0,9.823529,15.0,7.0,6.250000,0.750000,0.750000,0
2,112108,43633,train,1,1,3,1,4,4.000000,663,...,0.764706,4,167.0,9.823529,15.0,7.0,6.250000,0.750000,0.750000,0
3,112108,22035,train,1,1,3,1,4,6.000000,61241,...,0.764706,4,167.0,9.823529,15.0,7.0,6.250000,0.750000,0.750000,0
4,79431,19660,train,1,36,16,2,23,4.562500,57921,...,0.500000,23,3238.0,17.597826,30.0,2.0,8.391304,0.695652,0.727273,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8474656,117076,20539,prior,0,211941,1,1,1,5.000000,29,...,0.215909,10,1487.0,16.897727,30.0,1.0,9.500000,0.100000,0.100000,9
8474657,117076,35221,prior,0,211941,1,1,1,6.000000,48022,...,0.215909,10,1487.0,16.897727,30.0,1.0,9.500000,0.100000,0.100000,9
8474658,117076,12861,prior,0,211941,1,1,1,7.000000,21,...,0.215909,10,1487.0,16.897727,30.0,1.0,9.500000,0.100000,0.100000,9
8474659,175185,12023,prior,0,651501,1,23,23,6.000000,3554,...,0.572687,32,2242.0,9.876652,30.0,1.0,7.718750,0.031250,0.100000,9


In [35]:
train_df = full_df[['user_id','product_id','target']]

Creating DF that contains products from max_order-1 

In [10]:
test  = user_products[user_products['eval_set'] == 'train' ]
user_ids = test['user_id'].values
orders = user_products[user_products['user_id'].isin(user_ids)]

In [15]:
prior_orders = orders[orders.eval_set=='prior'][['user_id','product_id','reordered','order_number']]
prior_orders

,user_id,product_id,reordered,order_number
828824,202279,33120,1,3
828825,202279,28985,1,3
828826,202279,9327,0,3
828827,202279,45918,1,3
828828,202279,30035,0,3
...,...,...,...,...
33263298,175185,16797,0,23
33263299,175185,43352,1,23
33263300,175185,32700,1,23
33263301,175185,12023,0,23


In [18]:
prior_orders.groupby('user_id').order_number.max()

user_id
1         10
2         14
5          4
7         20
8          3
          ..
206199    19
206200    23
206203     5
206205     3
206209    13
Name: order_number, Length: 122607, dtype: int64

Create dataframe that has the most recent products ordered per user from the prior dataset

In [19]:
idx = prior_orders.groupby(['user_id'])['order_number'].transform(max) == prior_orders['order_number']
max_orders=prior_orders[idx]

In [20]:
max_orders.rename(columns={'reordered':'ordered_last_order'},inplace=True)
max_orders.sort_values('user_id')

/Users/tiffany.chen/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,user_id,product_id,ordered_last_order,order_number
25010093,1,38928,0,10
25010097,1,13032,1,10
25010098,1,12427,1,10
25010094,1,25133,1,10
25010092,1,39657,0,10
...,...,...,...,...
29064891,206209,31477,0,13
29064892,206209,6567,0,13
29064887,206209,16168,1,13
29064893,206209,22920,0,13


In [23]:
max_orders['predicted'] = 1
max_orders

<ipython-input-23-8db1e39a99eb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  max_orders['predicted'] = 1


,user_id,product_id,ordered_last_order,order_number,predicted
829043,59897,9755,1,19,1
829044,59897,31487,0,19,1
829045,59897,37510,1,19,1
829046,59897,14576,1,19,1
829047,59897,22105,0,19,1
...,...,...,...,...,...
33263096,123577,19678,0,6,1
33263097,123577,48775,1,6,1
33263098,123577,44047,0,6,1
33263244,103510,49187,0,30,1


Create dataframe with most recent order from training dataset we're using to predict on

In [ ]:
# train_orders = orders[orders.eval_set=='train'][['user_id','product_id','reordered','order_number']]
# # train_orders = orders[['user_id','product_id','reordered','order_number']]
# train_orders.sort_values('user_id')

Merge 2 dataframes, with training data as the base, and fill with 0 if product isn't ordered

In [36]:
baseline = pd.merge(train_df[['user_id','product_id','target']],max_orders[['user_id','product_id','predicted']],how='left',on=['user_id','product_id'])
baseline.isnull().sum()
baseline.predicted.fillna(0,inplace=True)
baseline.predicted.value_counts()
baseline.rename(columns={'target':'actual'},inplace=True)

# # baseline.astype({'actual': 'int32','predicted':'int32'}).dtypes
baseline


,user_id,product_id,actual,predicted
0,112108,49302,1,0.0
1,112108,11109,1,0.0
2,112108,43633,1,1.0
3,112108,22035,1,0.0
4,79431,19660,1,0.0
...,...,...,...,...
8474656,117076,20539,0,0.0
8474657,117076,35221,0,0.0
8474658,117076,12861,0,0.0
8474659,175185,12023,0,0.0


In [37]:
baseline.predicted.value_counts(normalize=True)

0.0    0.845619
1.0    0.154381
Name: predicted, dtype: float64

In [38]:
baseline.actual.value_counts(normalize=True)

0    0.9022
1    0.0978
Name: actual, dtype: float64

In [39]:
y_actual = baseline.actual
y_pred = baseline.predicted
cm = pd.DataFrame(metrics.confusion_matrix(y_true=y_actual, y_pred=y_pred), 
                  columns=['Negative Pred', 'Positive Pred'])
cm

,Negative Pred,Positive Pred
0,6716890,928947
1,449442,379382


In [43]:
tn = cm.iloc[0,0]
fn = cm.iloc[1,0]
fp = cm.iloc[0,1]
tp = cm.iloc[1,1]

accuracy = (tp+tn) / (tp+tn+fp+fn)
sensitivity = (tp) / (tp+fn)
precision= tp / (tp+fp) #proportion of the data points our model says was relevant actually were relevant
recall= tp/(tp+fn) #How many positives were predicted correctly
ppv = tp / (tp+fp)
print(accuracy, ',', sensitivity, ',', ppv,',',precision,',',recall)

0.8373517241574618 , 0.4577352972404274 , 0.28997446360968837 , 0.28997446360968837 , 0.4577352972404274


In [41]:
from sklearn.metrics import f1_score
f1_score(y_actual, y_pred)

0.3550349460239861

If the model were to predict all reorders, the model's accuracy would be 10%
Using only the user's last order, prediction precision and recall are 29%, with positive predictive value of 45%. 


## Baseline Model: Reorder Rate

In [44]:
full_df.head()
# prior_df.drop('Unnamed: 0',axis=1,inplace=True)

,user_id,product_id,eval_set,target,order_id,user_product_count,user_product_first_order,user_product_last_order,user_product_avg_basket_placement,product_ordered_vol,...,user_reorder_ratio,user_total_orders,user_lifetime_days,user_avg_days_between_orders,user_max_time_between_orders,user_min_time_between_orders,user_avg_cart_size,user_product_order_rate,user_product_reorder_rate,user_product_last_time_product_ordered
0,112108,49302,train,1,1,3,1,4,2.000000,170,...,0.764706,4,167.0,9.823529,15.0,7.0,6.250000,0.750000,0.750000,0
1,112108,11109,train,1,1,3,1,4,3.333333,4579,...,0.764706,4,167.0,9.823529,15.0,7.0,6.250000,0.750000,0.750000,0
2,112108,43633,train,1,1,3,1,4,4.000000,663,...,0.764706,4,167.0,9.823529,15.0,7.0,6.250000,0.750000,0.750000,0
3,112108,22035,train,1,1,3,1,4,6.000000,61241,...,0.764706,4,167.0,9.823529,15.0,7.0,6.250000,0.750000,0.750000,0
4,79431,19660,train,1,36,16,2,23,4.562500,57921,...,0.500000,23,3238.0,17.597826,30.0,2.0,8.391304,0.695652,0.727273,0


In [ ]:
# pred = full_df.groupby(['user_id','product_id']).agg(order_rate=('user_product_reorder_rate','mean')).reset_index()
# pred

In [46]:
baseline2 = full_df[['user_id','product_id','target','user_product_reorder_rate']]
baseline2.sort_values('user_id')

,user_id,product_id,target,user_product_reorder_rate
289403,1,196,1,1.000000
289407,1,39657,1,1.000000
289408,1,10258,1,1.000000
2489832,1,13176,0,0.200000
289406,1,26405,1,0.272727
...,...,...,...,...
8088318,206209,890,0,0.071429
8088317,206209,14901,0,0.071429
5918408,206209,40310,0,0.250000
3084441,206209,38167,0,0.333333


In [ ]:
# baseline_2 = pd.merge(train,pred,how='left',on=['user_id','product_id'])
# baseline_2

In [51]:
y_actual = baseline2.target

accuracy=[]
pos_pv=[]
sensitivity=[]

for i in np.arange(0.0,1.0,0.1):
    prob_thresh = i
    cm = pd.DataFrame(
        metrics.confusion_matrix(y_true=y_actual,y_pred=baseline2.user_product_reorder_rate > prob_thresh),
        columns=['Negative Pred', 'Positive Pred'])
    tn = cm.iloc[0,0]
    fn = cm.iloc[1,0]
    fp = cm.iloc[0,1]
    tp = cm.iloc[1,1]

    acc = (tp+tn) / (tp+tn+fp+fn)
#     tpr = tp / (tp+fn)
#     fpr = fp / (tn+fp)
    ppv = tp / (tp+fp)
    sens = (tp) / (tp+fn)


    accuracy.append(acc)
    pos_pv.append(ppv)
    sensitivity.append(sens)
    
    print(i, accuracy) 
    print(i, pos_pv)
    print(i, sensitivity)
    

0.0 [0.09780025419305857]
0.0 [0.09780025419305857]
0.0 [1.0]
0.1 [0.09780025419305857, 0.41316472717905767]
0.1 [0.09780025419305857, 0.14074648756605318]
0.1 [1.0, 0.9795046958099669]
0.2 [0.09780025419305857, 0.41316472717905767, 0.6580337549785177]
0.2 [0.09780025419305857, 0.14074648756605318, 0.21024174061307274]
0.2 [1.0, 0.9795046958099669, 0.9057290812042122]
0.30000000000000004 [0.09780025419305857, 0.41316472717905767, 0.6580337549785177, 0.7600724087960569]
0.30000000000000004 [0.09780025419305857, 0.14074648756605318, 0.21024174061307274, 0.26344068915753704]
0.30000000000000004 [1.0, 0.9795046958099669, 0.9057290812042122, 0.8091898883236972]
0.4 [0.09780025419305857, 0.41316472717905767, 0.6580337549785177, 0.7600724087960569, 0.8470369493245806]
0.4 [0.09780025419305857, 0.14074648756605318, 0.21024174061307274, 0.26344068915753704, 0.35453539547642876]
0.4 [1.0, 0.9795046958099669, 0.9057290812042122, 0.8091898883236972, 0.6873509936970937]
0.5 [0.09780025419305857, 0.